A short notebook to explore the spectrograms of the different species. Generates:
- a "regular" spectrogram using matplotlib,
- a mel-spectrogram using librosa,
- signal image reversed from the fourier transformations
- both for the full one minute in the sample, and for cropped signal data containing only the given sample window

I have no experience with audio analysis, spectrograms, etc., so any improvement suggestions are welcome.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import librosa
import librosa.display
import soundfile as sf
import os
import glob
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.fft import fft, fftfreq, rfft, rfftfreq, irfft

In [ ]:
trainfiles = glob.glob( '../input/rfcx-species-audio-detection/train/*.flac' )
testfiles = glob.glob( '../input/rfcx-species-audio-detection/test/*.flac' )
len(trainfiles), len(testfiles), trainfiles[0]

In [ ]:
df_train_tp = pd.read_csv( '../input/rfcx-species-audio-detection/train_tp.csv' )
df_train_tp["filepath"] = df_train_tp["recording_id"].apply(lambda rid: f"../input/rfcx-species-audio-detection/train/{rid}.flac")
df_train_tp['t_dif'] = df_train_tp['t_max'] - df_train_tp['t_min']
df_train_tp['f_dif'] = df_train_tp['f_max'] - df_train_tp['f_min']
df_train_tp["truepos"] = "1"

df_train_fp = pd.read_csv( '../input/rfcx-species-audio-detection/train_fp.csv' )
df_train_fp["filepath"] = df_train_fp["recording_id"].apply(lambda rid: f"../input/rfcx-species-audio-detection/train/{rid}.flac")
df_train_fp['t_dif'] = df_train_fp['t_max'] - df_train_fp['t_min']
df_train_fp['f_dif'] = df_train_fp['f_max'] - df_train_fp['f_min']
df_train_fp["truepos"] = 0

df_train_full = pd.concat([df_train_tp, df_train_fp])

df_train_tp.shape, df_train_fp.shape

In [ ]:
def collect_bird_metrics(df, species_id):
    df = df[df["species_id"] == species_id]
    df_metrics = df.describe()
    return df_metrics

all_metrics = []
for x in range(24):
    metrics = collect_bird_metrics(df_train_full, x)
    all_metrics.append(metrics)

In [ ]:
# Plot the spectrogram

sampling_frequency = 48000
visualize_fft_bins = False

def row_spectrogram(row, zoom, show_fft):
    dataitem, samplerate = sf.read(row["filepath"])
    start_time = row["t_min"]
    start_time = float(start_time)
    start_sample = start_time * sampling_frequency
    start_sample = int(start_sample)
    end_time = row["t_max"] 
    end_sample = end_time * sampling_frequency
    end_time = float(end_time)
    end_sample = int(end_sample)
    if zoom:
        dataitem = dataitem[start_sample:end_sample]
    freq_bottom = row["f_min"]
    freq_top = row["f_max"]

    if not zoom:
        plt.figure(figsize=(8,5))
        plt.title("Full one minute spectrogram with sample time and frequency highlighted")
        plt.axvspan(start_time, end_time, color='red', alpha=0.1)
    else:
        plt.figure(figsize=(5,5))
        plt.title("Sample time cropped spectrogram")
    plt.axhspan(freq_bottom, freq_top, color='red', alpha=0.3)
    powerSpectrum, freqenciesFound, time, imageAxis = plt.specgram(dataitem, Fs=sampling_frequency)
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.show()
    
    plt.figure(figsize=(8,5))
    if not zoom:
        plt.title('Full one minute Mel spectrogram, with full frequency range')
        mel_spectrogram = librosa.feature.melspectrogram(y=dataitem,
                                                             sr=samplerate,
                                                             n_mels=256,
                                                             fmax=samplerate/2,
                                                             hop_length=128)
    else:
        plt.title('Mel spectrogram for signal time and frequency range')
        #256 mels produced empty streaks, reduced to 128
        mel_spectrogram = librosa.feature.melspectrogram(y=dataitem,
                                                             sr=samplerate,
                                                             n_mels=128,
                                                             hop_length=64,
                                                             fmax=freq_top,
                                                             fmin=freq_bottom)

    librosa.display.specshow(librosa.power_to_db(mel_spectrogram,ref=np.max),
                              y_axis='mel', x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()

    if show_fft:
        bird_fft(dataitem, freq_bottom, freq_top, zoom)

def bird_spectrograms(df, species_id, idxs, zoom=False, show_fft=False):
    df = df[df["species_id"] == species_id]
    df = df.iloc[idxs]
    for index, row in df.iterrows():
        print(f"plotting row {index}:\n{row}")
        row_spectrogram(row, False, show_fft)
        if zoom:
            print(f"plotting (zoomed) sample area only: {row['t_min']}s to {row['t_max']}s")
            row_spectrogram(row, True, show_fft)
    
    
#https://realpython.com/python-scipy-fft/
def bird_fft(dataitem, freq_bottom, freq_top, zoom):
    yf = rfft(dataitem)
#    xf = rfftfreq(dataitem.shape[0], 1 / sampling_frequency * 100)
    xf = rfftfreq(dataitem.shape[0], 1 / sampling_frequency)

    if visualize_fft_bins:
        plt.figure(figsize=(8,5))
        if not zoom:
            plt.title("Full one minute FFT frequency weights")
        else:
            plt.title("Sample range FFT frequency weights")
        plt.plot(xf, np.abs(yf))
        plt.show()
    
    # The maximum frequency is half the sample rate (yes, I copied this from the internet :)
    points_per_freq = len(xf) / (sampling_frequency / 2)

    target_idx_bottom = int(points_per_freq * freq_bottom)
    target_idx_top = int(points_per_freq * freq_top)
    yf[target_idx_top:] = 0
    yf[:target_idx_bottom] = 0

    if visualize_fft_bins:
        plt.figure(figsize=(8,5))
        if not zoom:
            plt.title("Full FFT frequency weights for sample time")
        else:
            plt.title("Sample frequency range FFT frequency weights for sample time")
        plt.plot(xf, np.abs(yf))
        plt.show()

    #reverse the fft data back to signal without the filtered parts
    new_sig = irfft(yf)

    plt.figure(figsize=(8,5))
    if not zoom:
        plt.title("Reversed signal from sample FFT frequency weights, cropped to sample freq range")
    else:
        plt.title("Reversed signal from sample FFT frequency weights, cropped to sample time and freq range")
    plt.plot(new_sig)
    plt.show()
    
    #the following will draw a spectrogram for the filtered signal range(s)
#    plt.figure(figsize=(8,5))
#    powerSpectrum, freqenciesFound, time, imageAxis = plt.specgram(new_sig, Fs=sampling_frequency)
#    plt.title("Spectrogram from above FFT reversed signal")
#    plt.xlabel('Time')
#    plt.ylabel('Frequency')
#    plt.show()

In [ ]:
indices_to_plot = [0,1,2]

# Species 1

In [ ]:
x=0

In [ ]:
display(all_metrics[x])


In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 2

In [ ]:
x=1

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 3

In [ ]:
x = 2

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 4

In [ ]:
x = 3

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 5

In [ ]:
x = 4

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 6

In [ ]:
x = 5

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 7

In [ ]:
x = 6

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 8

In [ ]:
x = 7

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 9

In [ ]:
x = 8

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 10

In [ ]:
x = 9

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 11

In [ ]:
x = 10

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 12

In [ ]:
x = 11

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 13

In [ ]:
x = 12

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 14

In [ ]:
x = 13

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 15

In [ ]:
x = 14

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 16

In [ ]:
x = 15

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 17

In [ ]:
x = 16

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 18

In [ ]:
x = 17

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 19

In [ ]:
x = 18

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 20

In [ ]:
x = 19

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 21

In [ ]:
x = 20

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 22

In [ ]:
x = 21

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 23

In [ ]:
x = 22

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)

# Species 24

In [ ]:
x = 23

In [ ]:
display(all_metrics[x])

In [ ]:
bird_spectrograms(df_train_tp, x, indices_to_plot, True, True)